In [1]:
import csv
import cv2
import numpy as np
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
print(keras.__version__)

1.2.1


In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from random import shuffle
import sklearn

In [4]:
lines = []

In [5]:
with open('C:\Anaconda3\envs\CarND-Behavioral-Cloning-P3\Rec_Data\driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [6]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

In [7]:
def generator(samples, batch_size=32):
    correction = 0.15
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0,num_samples,batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            measurements = []
            augmented_images, augmented_measurements = [], []
            for batch_sample in batch_samples:
                for i in range(3):
                    source_path = batch_sample[i]
                    filename = source_path.split("\\")[-1]
                    current_path = 'C:\\Anaconda3\\envs\\CarND-Behavioral-Cloning-P3\\Rec_Data\\IMG\\' + filename
                    image = cv2.cvtColor(cv2.imread(current_path),cv2.COLOR_BGR2RGB)
                    if i==0:
                        measurement = float(batch_sample[3])
                    elif i==1:
                        measurement = float(batch_sample[3]) + correction
                    elif i==2:
                        measurement = float(batch_sample[3]) - correction
                    images.append(image)
                    measurements.append(measurement)
            for image, measurement in zip(images, measurements):
                augmented_images.append(image)
                augmented_measurements.append(measurement)
                augmented_images.append(cv2.flip(image,1))
                augmented_measurements.append(measurement*-1.0)
            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train,y_train)

In [8]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

#model = Sequential()
#model.add(Lambda(lambda x: (x/255.0)-0.5, input_shape=(160,320,3)))
#model.add(Flatten(input_shape=(160,320,3)))
#model.add(Dense(1))

model = Sequential()
model.add(Lambda(lambda x: (x/255.0)-0.5, input_shape=(160,320,3)))
model.add(Convolution2D(28,5,5,activation='relu',input_shape=(160,320,3) ))
model.add(MaxPooling2D()) #2x2 stride, valid padding, output size
model.add(Convolution2D(16,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(200))
#model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dense(1))

In [9]:
model = Sequential()
model.add(Lambda(lambda x: (x/255.0)-0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [10]:
model.compile(optimizer='adam', loss='mse')

In [11]:
model.fit_generator(train_generator, samples_per_epoch=7200,validation_data=validation_generator,nb_val_samples=1792, nb_epoch=3)

Epoch 1/3
7200/7200 [==============================] - 19s - loss: 0.0215 - val_loss: 0.0171
Epoch 2/3
7200/7200 [==============================] - 16s - loss: 0.0156 - val_loss: 0.0157
Epoch 3/3
7200/7200 [==============================] - 16s - loss: 0.0165 - val_loss: 0.0159


In [12]:
model.save('model_use_gen.h5')